In [23]:
import glob
import json
import os
import csv
import pandas as pd
import cv2
import numpy as np
from PIL import Image
from  matplotlib import pyplot as plt

In [24]:
# for split validation data
from numpy import random

In [25]:
file = open(os.path.join('./datasets/Rice/annotations', 'instances_train.json'), 'w')
data = {}

In [26]:
data["categories"] = []
data["images"] = []
data["annotations"] = []

In [27]:
data["categories"].append({"id":1,"name":"rice"})
print(data)

{'categories': [{'id': 1, 'name': 'rice'}], 'images': [], 'annotations': []}


### Training folder to json

In [28]:
image_path = sorted(glob.glob('./datasets/Rice/train/*'))
print(len(image_path))

980


In [29]:
for i in range(len(image_path)):
    path = image_path[i].split('/')
    img = cv2.imread(image_path[i])
    w, h = int(img.shape[1]), int(img.shape[0])
    data["images"].append({"id":i,"file_name":path[-1],"height":h, "width":w})
    if i < 5 :
        print("id",i,"file_name",path[-1])

id 0 file_name DSC080454 H:0 W:0.JPG
id 1 file_name DSC080454 H:0 W:1.JPG
id 2 file_name DSC080454 H:0 W:2.JPG
id 3 file_name DSC080454 H:0 W:3.JPG
id 4 file_name DSC080454 H:0 W:4.JPG


### Training annotations folder to json

In [30]:
csv_path = sorted(glob.glob('./datasets/Rice/train_labels/*.csv'))
print(len(csv_path))

980


In [31]:
count = 0
for i in range(len(csv_path)):
    #print(csv_path[i],len(df))
    try: 
        df = pd.read_csv(csv_path[i], header=None)
    except: 
        continue 
        
    img_path = csv_path[i][:-3]+'JPG'
    img_path = img_path.replace('train_labels','train')
    
    print(img_path, len(df))
    img = cv2.imread(img_path)
    img_w, img_h = int(img.shape[1]), int(img.shape[0])
    
    if len(df) > 200:
        w, h = 13, 13
        
    elif len(df) > 55:
        w, h = 15 , 15
    else : 
        w, h = 35, 35

    for k in range(len(df)):
        x,y = int(df.loc[k][0]), int(df.loc[k][1])
        # x,y,w,h
        bbox = [x,y,w,h]
        data["annotations"].append({"id":count, "image_id":i, "category_id":1, "iscrowd":0, "bbox":bbox})
        count+=1

./datasets/Rice/train/DSC080454 H:0 W:0.JPG 53
./datasets/Rice/train/DSC080454 H:0 W:1.JPG 34
./datasets/Rice/train/DSC080454 H:0 W:2.JPG 33
./datasets/Rice/train/DSC080454 H:0 W:3.JPG 27
./datasets/Rice/train/DSC080454 H:0 W:4.JPG 31
./datasets/Rice/train/DSC080454 H:0 W:5.JPG 21
./datasets/Rice/train/DSC080454 H:1 W:0.JPG 50
./datasets/Rice/train/DSC080454 H:1 W:1.JPG 30
./datasets/Rice/train/DSC080454 H:1 W:2.JPG 32
./datasets/Rice/train/DSC080454 H:1 W:3.JPG 31
./datasets/Rice/train/DSC080454 H:1 W:4.JPG 29
./datasets/Rice/train/DSC080454 H:1 W:5.JPG 26
./datasets/Rice/train/DSC080454 H:2 W:0.JPG 48
./datasets/Rice/train/DSC080454 H:2 W:1.JPG 30
./datasets/Rice/train/DSC080454 H:2 W:2.JPG 34
./datasets/Rice/train/DSC080454 H:2 W:3.JPG 31
./datasets/Rice/train/DSC080454 H:2 W:4.JPG 33
./datasets/Rice/train/DSC080454 H:2 W:5.JPG 28
./datasets/Rice/train/DSC080454 H:3 W:0.JPG 36
./datasets/Rice/train/DSC080454 H:3 W:1.JPG 29
./datasets/Rice/train/DSC080454 H:3 W:2.JPG 18
./datasets/Ri

In [32]:
print("annotations count :",count)

annotations count : 45717


In [33]:
json.dump(data, file)
file.close()

### Observe All data

In [34]:
annotation_path = './datasets/Rice/annotations/instances_train.json'
f = open(annotation_path)
data = json.load(f)

for read_img_id in range(0,len(image_path)):
    img_id, img_name = data['images'][read_img_id]['id'], data['images'][read_img_id]['file_name']
    #print(img_id, img_name)
    annotations_len = len(data['annotations'])
    bboxs = []

    for i in range(0,annotations_len):
        if data['annotations'][i]['image_id'] != img_id : 
            continue
        bboxs.append(data['annotations'][i]['bbox'])

    img_path = os.path.join('./datasets/Rice/train',img_name)
    img = Image.open(img_path, mode='r').convert('RGB')

    out_img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

    for i in range(len(bboxs)):
        x,y,w,h = int(bboxs[i][0]), int(bboxs[i][1]), int(bboxs[i][2]), int(bboxs[i][3])
        x1,y1,x2,y2 = x-w, y-h, x+w, y+h
        out_img = cv2.rectangle(out_img, (x1,y1), (x2,y2), (0, 0, 255), 2)

    out_img = out_img[:,:,::-1] #BGR->RGB
    plt.imsave('./observe_data/'+img_name[:-4]+'_observe.jpg',out_img)

f.close()